In [1]:
install.packages("imputeTestbench")

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
# load packages, set seed
library(tidyverse)
library(imputeTestbench)
set.seed(123)


Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Warning message:
“replacing previous import ‘forecast::autolayer’ by ‘ggplot2::autolayer’ when loading ‘imputeTestbench’”

In [3]:
dfDados=readRDS("../plumber/dfDadosIoTa.norm.rds")
  nrm = dfDados$dados[which(dfDados$dia==1)]
  aust = dfDados$dados[which(dfDados$dia==2)]
  nott = dfDados$dados[which(dfDados$dia==3)]
# create nested list of datasets
dats <- list(nrm, aust, nott) %>% enframe ('nms', 'dataIn')
# create parameters to vary with imputeTestbench
prms <- crossing(smps = c('mcar', 'mar'), blck = c(20, 100), nms = dats$nms) %>%mutate( # blck does not matter for mcar
    blck = ifelse(smps %in% 'mcar', NA, blck)
  ) %>% unique
# join parameters with data
# map parameter and data to impute_errors
ests <- prms %>%
  left_join(dats, by = 'nms') %>%
  mutate(
    est = pmap(
      list(smps = smps, blck = blck, dataIn = dataIn),
      .f = impute_errors,
      errorParameter = 'mape'
      )
    )